<a href="https://colab.research.google.com/github/Jonchyk/Datamgmt/blob/main/PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

!pip install pyreadstat

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg
##https://www.marktechpost.com/2023/10/20/6-magic-commands-for-jupyter-notebooks-in-python-data-science/
#%%latex
#%ai
#%run
#%writefile
#%history -n

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

#sometimes have to install library which you get from https://pypi.org/
#!pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.2 MB/s eta 0:00:00


In [3]:
#Background
#For PS1, I will be running three datasets - focused on early grade learning by region and language, pop change by region, and per capita income by region
#dataset 1: Foundational Learning Asessment -  an SPSS file found from UNICEF's Multi Cluster study in Kyrgystan, 2023 - https://mics.unicef.org/news/just-released-kyrgyzstan-2023-mics-survey-findings-snapshots-and-datasets
#will read this in, review, recode columns, and review the sample that I have to work with as well as make a plan of attack for creating scores by region and urban vs rural type
#The second dataset is population in millions in Kyrgyzstan. I'll run it, clean it, and calculate the percentage change per region year over year.
#purpose of this dataset is to understand demographic changes within Kyrgyzstan, leading up to the MCIS
#i'll also calculate a compound growth rate, to get a grasp on each region's change leading up to 2023.
#The Final Dataset is going to just look at the average income per region. Nothing fancy


In [4]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb' -O mics.sav


--2025-01-27 23:40:20--  https://docs.google.com/uc?export=download&id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb
Resolving docs.google.com (docs.google.com)... 173.194.194.102, 173.194.194.113, 173.194.194.138, ...
Connecting to docs.google.com (docs.google.com)|173.194.194.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb&export=download [following]
--2025-01-27 23:40:21--  https://drive.usercontent.google.com/download?id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.193.132, 2607:f8b0:4001:c0f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.193.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1807033 (1.7M) [application/octet-stream]
Saving to: ‘mics.sav’

mics.sav            100%[===================>]   1.72M  --.-KB/s    i

In [5]:
#read spss file
df = pd.read_spss("mics.sav")
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns', None)  # No limit on columns
#let's take a peek at this data set.
df.head(10)
#Let's specifically look at our geographic breakdown, since this is important
df["stratum"].value_counts()

,HH1,HH2,LN,FS1,FS2,FS3,FSINT,FS4,FS5,FS6,FS7D,FS7M,FS7Y,FS9,FS10,FS17,FS8H,FS8M,FS11H,FS11M,FS12,FS13,FS14,FS15,FSHINT,FSFIN,FS15A,FS15C,FS15D,CB2M,CB2Y,CB3,CB4,CB5A,CB5B,CB6,CB7,CB8A,CB8B,CB9,CB10A,CB10B,CL1A,CL1B,CL1C,CL1X,CL3,CL4,CL5,CL6A,CL6B,CL6C,CL6D,CL6E,CL6X,CL7,CL8,CL9,CL10,CL11A,CL11B,CL11C,CL11D,CL11E,CL11F,CL11X,CL13,FCD2A,FCD2B,FCD2C,FCD2D,FCD2E,FCD2F,FCD2G,FCD2H,FCD2I,FCD2J,FCD2K,FCD3,FCD4,FCD5,FCF1,FCF2,FCF3,FCF6,FCF8,FCF10,FCF11,FCF12,FCF13,FCF14,FCF15,FCF16,FCF17,FCF18,FCF19,FCF20,FCF21,FCF22,FCF23,FCF24,FCF25,FCF26,PR3,PR5,PR6,PR7,PR8,PR9A,PR9B,PR10,PR11A,PR11B,PR12A,PR12B,PR12C,PR12X,PR13,PR15,FL1,FL2H,FL2M,FL3,FL4A,FL4B,FL4C,FL6A,FL6B,FL7,FL9,FL10,FL10C,FL13,FL14,FL15,FL16,FL17,FL18,FL19INTRO,FL19W1,FL19W2,FL19W3,FL19W4,FL19W5,FL19W6,FL19W7,FL19W8,FL19W9,FL19W10,FL19W11,FL19W12,FL19W13,FL19W14,FL19W15,FL19W16,FL19W17,FL19W18,FL19W19,FL19W20,FL19W21,FL19W22,FL19W23,FL19W24,FL19W25,FL19W26,FL19W27,FL19W28,FL19W29,FL19W30,FL19W31,FL19W32,FL19W33,FL19W34,FL19W35,FL19W36,FL19W37,FL19W38,FL19W39,FL19W40,FL19W41,FL19W42,FL19W43,FL19W44,FL19W45,FL19W46,FL19W47,FL19W48,FL19W49,FL19W50,FL19W51,FL19W52,FL19W53,FL19W54,FL19W55,FL19W56,FL19W57,FL19W58,FL19W59,FL19W60,FL19W61,FL19W62,FL19W63,FL19W64,FL19W65,FL19W66,FL19W67,FL19W68,FL19W69,FL19W70,FL19W71,FL19W72,FL19END,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL21D,FL21G,FL21H,FL21I,FL21J,FL21K,FL21L,FL21OINTRO,FL21OW1,FL21OW2,FL21OW3,FL21OW4,FL21OW5,FL21OW6,FL21OW7,FL21OW8,FL21OW9,FL21OW10,FL21OW11,FL21OW12,FL21OW13,FL21OW14,FL21OW15,FL21OW16,FL21OW17,FL21OW18,FL21OW19,FL21OW20,FL21OW21,FL21OW22,FL21OW23,FL21OW24,FL21OW25,FL21OW26,FL21OW27,FL21OW28,FL21OW29,FL21OW30,FL21OW31,FL21OW32,FL21OW33,FL21OW34,FL21OW35,FL21OW36,FL21OW37,FL21OW38,FL21OW39,FL21OW40,FL21OW41,FL21OW42,FL21OW43,FL21OW44,FL21OW45,FL21OW46,FL21OW47,FL21OW48,FL21OW49,FL21OW50,FL21OW51,FL21OW52,FL21OW53,FL21OW54,FL21OEND,FL21PA,FL21PB,FL22A,FL22B,FL22C,FL22D,FL22E,FL23A,FL23B,FL23C,FL23D,FL23E,FL23F,FL24A,FL24B,FL24C,FL24D,FL24E,FL25A,FL25B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,FSAGE,FSDOI,FSDOB,fshweight,fsweight,melevel,fselevel,schage,fsdisability,ethnicity,caretakerdis,wscore,windex5,windex10,wscoreu,windex5u,windex10u,wscorer,windex5r,windex10r,PSU,stratum
0,1.0,8.0,3.0,1.0,8.0,3.0,35.0,2.0,35.0,30.0,22.0,11.0,2023.0,"NO, FIRST INTERVIEW",YES,COMPLETED,17.0,30.0,17.0,53.0,KYRGYZ,KYRGYZ,KYRGYZ,"NO, NOT USED",35.0,SAVE QUESTIONNAIRE AND FINISH,NO,NO,NaN,MAY,2011.0,12.0,YES,BASIC SECONDARY,GRADE 6,NO,YES,BASIC SECONDARY,GRADE 6,YES,BASIC SECONDARY,GRADE 5,NO,NO,NO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,NO,NaN,YES,NO,YES,NO,NO,NO,YES,5.0,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NO,NaN,NO,NO,NO,NO,NO DIFFICULTY,NO DIFFICULTY,NaN,NaN,NaN,NaN,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,A FEW TIMES A YEAR,NEVER,TEN OR MORE BOOKS,YES,NO,YES,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NaN,"YES, PERMISSION IS GIVEN",17.0,30.0,YES,You are not alone with the child unless he/she...,You have engaged the child in conversation and...,"The child is sat comfortably, able to use the ...",YES,NO,RUSSIAN,RUSSIAN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONTINUE,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,NOT REACHED,CONTINUE,71.0,0.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRE

,count
stratum,
Naryn - rural,480
Talas - rural,443
Osh city - urban,430
Osh - rural,387
Chui - rural,360
Batken - rural,327
Bishkek city - urban,308
Jala-Abad - rural,296
Issyk-kul - rural,284


In [6]:
#let's get a full list of columns
print(list(df.columns))

['HH1', 'HH2', 'LN', 'FS1', 'FS2', 'FS3', 'FSINT', 'FS4', 'FS5', 'FS6', 'FS7D', 'FS7M', 'FS7Y', 'FS9', 'FS10', 'FS17', 'FS8H', 'FS8M', 'FS11H', 'FS11M', 'FS12', 'FS13', 'FS14', 'FS15', 'FSHINT', 'FSFIN', 'FS15A', 'FS15C', 'FS15D', 'CB2M', 'CB2Y', 'CB3', 'CB4', 'CB5A', 'CB5B', 'CB6', 'CB7', 'CB8A', 'CB8B', 'CB9', 'CB10A', 'CB10B', 'CL1A', 'CL1B', 'CL1C', 'CL1X', 'CL3', 'CL4', 'CL5', 'CL6A', 'CL6B', 'CL6C', 'CL6D', 'CL6E', 'CL6X', 'CL7', 'CL8', 'CL9', 'CL10', 'CL11A', 'CL11B', 'CL11C', 'CL11D', 'CL11E', 'CL11F', 'CL11X', 'CL13', 'FCD2A', 'FCD2B', 'FCD2C', 'FCD2D', 'FCD2E', 'FCD2F', 'FCD2G', 'FCD2H', 'FCD2I', 'FCD2J', 'FCD2K', 'FCD3', 'FCD4', 'FCD5', 'FCF1', 'FCF2', 'FCF3', 'FCF6', 'FCF8', 'FCF10', 'FCF11', 'FCF12', 'FCF13', 'FCF14', 'FCF15', 'FCF16', 'FCF17', 'FCF18', 'FCF19', 'FCF20', 'FCF21', 'FCF22', 'FCF23', 'FCF24', 'FCF25', 'FCF26', 'PR3', 'PR5', 'PR6', 'PR7', 'PR8', 'PR9A', 'PR9B', 'PR10', 'PR11A', 'PR11B', 'PR12A', 'PR12B', 'PR12C', 'PR12X', 'PR13', 'PR15', 'FL1', 'FL2H', 'FL2M',

In [23]:
# Convert FL20A and FL20B to numeric from categorical to get around the errors; now, let's get my percentage of words attempted vs incorrect words for a "reading" score.
df['FL20A'] = pd.to_numeric(df['FL20A'], errors='coerce')
df['FL20B'] = pd.to_numeric(df['FL20B'], errors='coerce')# Calculate the percentage of correct words
df['wrdprctcorrect'] = ((df['FL20A'] - df['FL20B']) / df['FL20A']) * 100

# Oops. The above equation will come out all sorts of wrong if I don't ensure that someone with a perfect score isn't dividing by 0. values for cases where FL20B is 0 (all correct attempts)
df.loc[df['FL20B'] == 0, 'wrdprctcorrect'] = 100



In [52]:
#Now I need to get something else...what are the values in these five columns? FL21BA	FL21BB	FL21BC	FL21BD	FL21BE
df[['FL21BA', 'FL21BB',	'FL21BC'	,'FL21BD'	,'FL21BE']]


,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE
0,1,1,1,1,1
2,1,1,1,1,1
5,1,1,1,1,1
6,1,1,1,1,1
8,1,1,1,1,1
...,...,...,...,...,...
3924,1,1,1,1,1
3925,1,1,1,0,1
3927,1,1,1,1,1
3930,1,1,1,1,1


In [50]:
#ok. Now that i've done this. I've identified all of the "columns" that need to be kept. it's a lot. Thankfully, we are keeping all of the FL codes (foundational learning)
#Let's drop all non necessary columns here, keeping language/geographic/ethnicity, grade, gender, age, questions on child labor, natural disasters, and the full foundational assessment questionaire
df = df.loc[:, df.columns.isin(['FS10', 'FS12', 'FS13', 'FS14', 'CB3', 'CB5B' , 'wrdprctcorrect', 'FL3' , 'FL7','FL9', 'FL20A', 'FL20B','FL21A', 'FL21BA', 'FL21BB', 'FL21BC', 'FL21BD', 'FL21BE', 'FL23A', 'FL23B', 'FL25C', 'FL25D', 'FL25E', 'FL26', 'FL26A', 'FL26B', 'FL26C', 'FL26D', 'FL26E', 'FL27A', 'FL27B', 'FL27C', 'FL27D', 'FL27E', 'FL28', 'ST1LANG', 'ST1WNUM', 'ST2LANG', 'ST2WNUM', 'HH4', 'HH6', 'HH7', 'HH52', 'HL4', 'ED5A', 'ED5B', 'ethnicity','stratum'])]

In [51]:
df.head(1)

,FS10,FS12,FS13,FS14,CB3,CB5B,FL3,FL7,FL9,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL23A,FL23B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,ethnicity,stratum,wrdprctcorrect
0,YES,KYRGYZ,KYRGYZ,KYRGYZ,12.0,GRADE 6,YES,RUSSIAN,RUSSIAN,71.0,0.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,1.0,MALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 2,Russian,Issyk-kul - urban,100.0


In [26]:
#let's get this down to 7 to 10 years old, since the data collected is for ages 7-14, but is a G2 foundational test. Wow!
df_age = df[(df["CB3"] > 6) & (df["CB3"] < 11)]
#how many respondents do we have now?
df_age["stratum"].value_counts()

,count
stratum,
Chui - rural,99
Osh - rural,97
Naryn - rural,94
Talas - rural,87
Batken - rural,79
Osh city - urban,79
Issyk-kul - rural,78
Bishkek city - urban,68
Jala-Abad - rural,54


In [27]:
#further, let's see how many of these children consented to taking the test.
df_age["FL3"].value_counts()

,count
FL3,
YES,914
NO / NOT ASKED,0


In [28]:
#1219 said yes! Of those 1219, how many do we have in each "stratum" now?
df_age[["stratum", "FL3"]].value_counts(normalize=False)
#ok! This looks like it's enough data to work with.  While some of the urban smaples are a bit low, every sample per region is >30!


,,count
stratum,FL3,
Chui - rural,YES,99
Osh - rural,YES,97
Naryn - rural,YES,94
Talas - rural,YES,87
Batken - rural,YES,79
Osh city - urban,YES,79
Issyk-kul - rural,YES,78
Bishkek city - urban,YES,68
Jala-Abad - rural,YES,54


In [29]:
#run the dataframe and double check that we have only the columns listed above and all looks ok.
df_age

,FS10,FS12,FS13,FS14,CB3,CB5B,FL3,FL7,FL9,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL23A,FL23B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,ethnicity,wscore,windex5,windex10,stratum,wrdprctcorrect
2,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 1,YES,KYRGYZ,RUSSIAN,71.0,4.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,3.0,FEMALE,HIGHER,GRADE 4,Kyrgyz,0.390934,Fourth,8th decile,Issyk-kul - urban,94.366197
8,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,2.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),INCORRECT,CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,2.0,FEMALE,HIGHER,GRADE 5,Other ethnicity,1.751447,Richest,10th decile,Issyk-kul - urban,97.183099
17,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 4,YES,KYRGYZ,RUSSIAN,71.0,5.0,1,1,1,1,0,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,KYRGYZ,47.0,30.0,URBAN,Issyk-kul,1.0,FEMALE,COMPLETE SECONDARY,GRADE 11,Kyrgyz,0.442114,Fourth,8th decile,Issyk-kul - urban,92.957746
18,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 4,YES,KYRGYZ,KYRGYZ,59.0,5.0,1,1,1,1,0,CORRECT(9),INCORRECT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETED,KYRGYZ,59.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,3.0,FEMALE,HIGHER,GRADE 5,Kyrgyz,0.673772,Richest,9th decile,Issyk-kul - urban,91.525424
19,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 3,YES,KYRGYZ,KYRGYZ,59.0,0.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,KYRGYZ,59.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,1.0,MALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 1,Kyrgyz,0.812186,Richest,9th decile,Issyk-kul - urban,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3921,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,0.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,90.0,URBAN,Osh c,2.0,FEMALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Kyrgyz,-0.537386,Second,4th decile,Osh city - urban,100.000000
3923,YES,KYRGYZ,KYRGYZ,KYRGYZ,8.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,0.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,90.0,URBAN,Osh c,2.0,MALE,HIGHER,GRADE 5,Kyrgyz,0.506142,Richest,9th decile,Osh city - urban,100.000000
3924,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 4,YES,KYRGYZ,KYRGYZ,59.0,0.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,KYRGYZ,59.0,WAS NOT NECESSARY TO OFFER,NaN,90.0,URBAN,Osh c,4.0,FEMALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Kyrgyz,1.101478,Richest,9th decile,Osh city - urban,100.000000
3925,YES,RUSSIAN,KYRGYZ,KYRGYZ,10.0,GRADE 5,YES,KYRGYZ,RUSSIAN,71.0,0.0,1,1,1,0,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),INCORRECT,CORRECT (3),CONTINUE,NaN,CORRECT (

In [30]:
#what are the counts for each grade.
df_age["CB5B"].value_counts( normalize=False)

,count
CB5B,
GRADE 4,254
GRADE 3,246
GRADE 2,221
GRADE 5,95
GRADE 1,91
GRADE 6,4
GRADE 10,0
GRADE 11,0
GRADE 7,0


In [31]:
#I want to filter out everyone that isn't in Grade 2 or Grade 3, since this is a grade 2 foundational test
filtered_df = df_age[df_age["CB5B"].isin(["GRADE 2", "GRADE 3"])]
#let's run this and see the count by stratum. Is this enough to be representative? Maybe ok if done at regional (non urban/rural level)
filtered_df["stratum"].value_counts( normalize=False)


,count
stratum,
Osh - rural,56
Chui - rural,48
Naryn - rural,47
Talas - rural,42
Issyk-kul - rural,39
Batken - rural,38
Bishkek city - urban,35
Osh city - urban,34
Jala-Abad - rural,28


In [33]:
filtered_df["FL20B"]

,FL20B
8,2.0
19,0.0
29,0.0
47,0.0
49,0.0
...,...
3903,0.0
3912,0.0
3921,0.0
3923,0.0


In [32]:
#Another question remains; the questionaire asks children what language they speak at home, versus hear at school (FL7, FL9). How does those counts come about?
pd.crosstab(df_age['FL7'], df_age['FL9'])
#by basis of ethnicity compared to language of instruction
pd.crosstab(df_age['ethnicity'], df_age['FL9'])
#by basis of ethnicity compared to language at home
pd.crosstab(df_age['ethnicity'], df_age['FL7'])



FL9,KYRGYZ,RUSSIAN,TAJIK,UZBEK
FL7,,,,
15.0,0,0,2,0
16.0,0,3,1,0
17.0,0,1,0,0
KYRGYZ,538,150,0,0
OTHER,1,1,0,0
RUSSIAN,4,114,0,0
TAJIK,0,0,1,0
UZBEK,8,47,0,40


FL9,KYRGYZ,RUSSIAN,TAJIK,UZBEK
ethnicity,,,,
Kyrgyz,535,199,0,2
Other ethnicity,8,22,4,1
Russian,0,32,0,0
Uzbek,8,63,0,37


FL7,15.0,16.0,17.0,KYRGYZ,OTHER,RUSSIAN,TAJIK,UZBEK
ethnicity,,,,,,,,
Kyrgyz,0,0,0,674,0,62,0,2
Other ethnicity,2,4,1,9,2,14,1,3
Russian,0,0,0,0,0,32,0,0
Uzbek,0,0,0,6,0,11,0,91


In [34]:
#Let's rename certain columns, not all of them, but a few critical pieces - still working on this
df_age.rename(columns={'FS10': 'consent', 'FS17': 'completed', 'FS12': 'language_questionaire', 'FS13': 'language_interview', 'FS14': 'native_language', 'CB5': 'highest_grade'}, inplace=False)

,consent,language_questionaire,language_interview,native_language,CB3,CB5B,FL3,FL7,FL9,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL23A,FL23B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,ethnicity,wscore,windex5,windex10,stratum,wrdprctcorrect
2,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 1,YES,KYRGYZ,RUSSIAN,71.0,4.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,3.0,FEMALE,HIGHER,GRADE 4,Kyrgyz,0.390934,Fourth,8th decile,Issyk-kul - urban,94.366197
8,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,2.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),INCORRECT,CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,2.0,FEMALE,HIGHER,GRADE 5,Other ethnicity,1.751447,Richest,10th decile,Issyk-kul - urban,97.183099
17,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 4,YES,KYRGYZ,RUSSIAN,71.0,5.0,1,1,1,1,0,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,KYRGYZ,47.0,30.0,URBAN,Issyk-kul,1.0,FEMALE,COMPLETE SECONDARY,GRADE 11,Kyrgyz,0.442114,Fourth,8th decile,Issyk-kul - urban,92.957746
18,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 4,YES,KYRGYZ,KYRGYZ,59.0,5.0,1,1,1,1,0,CORRECT(9),INCORRECT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETED,KYRGYZ,59.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,3.0,FEMALE,HIGHER,GRADE 5,Kyrgyz,0.673772,Richest,9th decile,Issyk-kul - urban,91.525424
19,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 3,YES,KYRGYZ,KYRGYZ,59.0,0.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,KYRGYZ,59.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,1.0,MALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 1,Kyrgyz,0.812186,Richest,9th decile,Issyk-kul - urban,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3921,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,0.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,90.0,URBAN,Osh c,2.0,FEMALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Kyrgyz,-0.537386,Second,4th decile,Osh city - urban,100.000000
3923,YES,KYRGYZ,KYRGYZ,KYRGYZ,8.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,0.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,90.0,URBAN,Osh c,2.0,MALE,HIGHER,GRADE 5,Kyrgyz,0.506142,Richest,9th decile,Osh city - urban,100.000000
3924,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 4,YES,KYRGYZ,KYRGYZ,59.0,0.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,KYRGYZ,59.0,WAS NOT NECESSARY TO OFFER,NaN,90.0,URBAN,Osh c,4.0,FEMALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Kyrgyz,1.101478,Richest,9th decile,Osh city - urban,100.000000
3925,YES,RUSSIAN,KYRGYZ,KYRGYZ,10.0,GRADE 5,YES,KYRGYZ,RUSSIAN,71.0,0.0,1,1,1,0,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19)

In [35]:
#check columns rename went through, use false first, then rerun below with true
df_age.head(1)

,FS10,FS12,FS13,FS14,CB3,CB5B,FL3,FL7,FL9,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL23A,FL23B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,ethnicity,wscore,windex5,windex10,stratum,wrdprctcorrect
2,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 1,YES,KYRGYZ,RUSSIAN,71.0,4.0,1,1,1,1,1,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,3.0,FEMALE,HIGHER,GRADE 4,Kyrgyz,0.390934,Fourth,8th decile,Issyk-kul - urban,94.366197


In [36]:
#repeat df rename function but htis time with inplace = true

In [37]:
#NEXT STEPS for this Dataset:
#ok, this is interesting. I'll need to look into this deeper when thinking about how to compare and contrast. Did students read in their language of instruction, or language at home?
#For the next PS, will's double check and create a column that returns 'languageathome1" or "languageofinstruction1", if the child chose to read a storyu, was it based on language at home or language of instruction?
#Will likely drop the second story. We'll assess foundational knowledge based on percentage of reading and math.
#Then run the average score in each of those  categories, based on the stratum (Oblast, rural/urban)
#then compare results by age, and by gender


In [38]:
# In the future can do a merge and roll up, for now, let's keep as is. Need to export this to a CSV file, right?

In [39]:
#now to read in a second dataset - let's grab some data from UNGLOBAL Data lab by region, it's quite clean and ready to go!
!wget --no-check-certificate 'https://docs.google.com/spreadsheets/d/1V9cMDdPdcf19ahKqUoT2uxJxsXas-qGYLSBD3QG_rL8/export?format=csv' -O GDL-pop-data.csv

pop = pd.read_csv('/content/GDL-pop-data.csv')
pop.head(1)

--2025-01-27 23:51:48--  https://docs.google.com/spreadsheets/d/1V9cMDdPdcf19ahKqUoT2uxJxsXas-qGYLSBD3QG_rL8/export?format=csv
Resolving docs.google.com (docs.google.com)... 142.251.183.101, 142.251.183.139, 142.251.183.113, ...
Connecting to docs.google.com (docs.google.com)|142.251.183.101|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-10-7s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/mbh8iaejf120ea2hkn73g4clkk/1738021905000/113781219181981321798/*/1V9cMDdPdcf19ahKqUoT2uxJxsXas-qGYLSBD3QG_rL8?format=csv [following]
--2025-01-27 23:51:48--  https://doc-10-7s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/mbh8iaejf120ea2hkn73g4clkk/1738021905000/113781219181981321798/*/1V9cMDdPdcf19ahKqUoT2uxJxsXas-qGYLSBD3QG_rL8?format=csv
Resolving doc-10-7s-sheets.googleusercontent.com (doc-10-7s-sheets.googleusercontent.com)... 74.125.202.132, 2607:f8b0:4001:c06::84
Connecting to doc-10-7s-sheets.googleuserc

,Country,Continent,ISOCode,Level,GDLCODE,Region,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kyrgyz Republic,Asia/Pacific,KGZ,National,KGZt,Total,NaN,NaN,4.52,4.58,4.64,4.7,4.75,4.81,4.87,4.93,4.99,5.04,5.1,5.16,5.22,5.28,5.35,5.41,5.48,5.54,5.61,5.72,5.84,5.96,6.08,6.2,6.32,6.44,6.57,6.69,NaN,NaN


In [40]:
pop.drop(columns=['Country', 'Continent', 'ISOCode', 'Level', 'GDLCODE'], inplace=True)
pop.head(10)

,Region,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Total,NaN,NaN,4.52,4.58,4.64,4.70,4.75,4.81,4.87,4.93,4.99,5.04,5.10,5.16,5.22,5.28,5.35,5.41,5.48,5.54,5.61,5.72,5.84,5.96,6.08,6.20,6.32,6.44,6.57,6.69,NaN,NaN
1,Batken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.42,0.42,0.42,0.42,0.42,0.42,0.43,0.43,0.43,0.43,0.45,0.48,0.47,0.47,0.46,0.46,0.46,0.45,0.45,NaN,NaN
2,Bishkek,NaN,NaN,0.42,0.46,0.50,0.54,0.58,0.62,0.66,0.70,0.74,0.78,0.82,0.86,0.90,0.88,0.87,0.85,0.84,0.83,0.81,0.78,0.75,0.86,0.97,1.08,1.19,1.31,1.42,1.53,NaN,NaN
3,Chuy,NaN,NaN,0.81,0.81,0.81,0.81,0.81,0.81,0.81,0.81,0.80,0.80,0.80,0.80,0.80,0.83,0.86,0.89,0.91,0.94,0.97,1.01,1.04,1.03,1.01,0.99,0.98,0.96,0.95,0.93,NaN,NaN
4,Issyk-Kul,NaN,NaN,0.46,0.45,0.45,0.45,0.44,0.44,0.43,0.43,0.42,0.42,0.42,0.41,0.41,0.42,0.44,0.45,0.47,0.48,0.50,0.47,0.44,0.45,0.47,0.48,0.50,0.51,0.53,0.54,NaN,NaN
5,Jalal-Abad,NaN,NaN,0.82,0.83,0.84,0.86,0.87,0.88,0.89,0.91,0.92,0.93,0.94,0.96,0.97,0.97,0.98,0.98,0.98,0.98,0.99,1.07,1.15,1.13,1.10,1.07,1.05,1.02,0.99,0.96,NaN,NaN
6,Naryn,NaN,NaN,0.27,0.27,0.27,0.27,0.26,0.26,0.26,0.26,0.25,0.25,0.25,0.25,0.24,0.25,0.25,0.25,0.25,0.26,0.26,0.27,0.28,0.28,0.29,0.29,0.30,0.30,0.31,0.31,NaN,NaN
7,Osh,NaN,NaN,1.68,1.64,1.61,1.58,1.54,1.51,1.47,1.44,1.41,1.37,1.34,1.30,1.27,1.29,1.30,1.32,1.34,1.35,1.37,1.39,1.41,1.45,1.49,1.53,1.57,1.62,1.66,1.70,NaN,NaN
8,Talas,NaN,NaN,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.21,0.21,0.21,0.21,0.21,0.21,0.22,0.23,0.25,0.26,0.27,0.28,0.29,0.30,0.29,0.29,0.28,0.28,0.27,0.27,0.26,NaN,NaN


In [41]:
# Convert the relevant columns (excluding the first column 'Region') to numeric
for col in pop.columns[1:]:  # Skip the first column (Region)
    pop[col] = pd.to_numeric(pop[col], errors='coerce')  # Replace non-numeric values with NaN

# Now calculate the percentage change across columns (years), skipping NaN values
pop_pct_change = pop.drop('Region', axis=1).pct_change(axis='columns', fill_method=None) * 100

# Reattach the 'Region' column at the start (add it as the first column)
pop_pct_change.insert(0, 'Region', pop['Region'])

# Display the first 10 rows of the percentage change DataFrame with 'Region' column at the start
pop_pct_change.head(10)

,Region,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Total,NaN,NaN,NaN,1.327434,1.310044,1.293103,1.063830,1.263158,1.247401,1.232033,1.217039,1.002004,1.190476,1.176471,1.162791,1.149425,1.325758,1.121495,1.293900,1.094891,1.263538,1.960784,2.097902,2.054795,2.013423,1.973684,1.935484,1.898734,2.018634,1.826484,NaN,NaN
1,Batken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,2.380952,0.000000,0.000000,0.000000,4.651163,6.666667,-2.083333,0.000000,-2.127660,0.000000,0.000000,-2.173913,0.000000,NaN,NaN
2,Bishkek,NaN,NaN,NaN,9.523810,8.695652,8.000000,7.407407,6.896552,6.451613,6.060606,5.714286,5.405405,5.128205,4.878049,4.651163,-2.222222,-1.136364,-2.298851,-1.176471,-1.190476,-2.409639,-3.703704,-3.846154,14.666667,12.790698,11.340206,10.185185,10.084034,8.396947,7.746479,NaN,NaN
3,Chuy,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.234568,0.000000,0.000000,0.000000,0.000000,3.750000,3.614458,3.488372,2.247191,3.296703,3.191489,4.123711,2.970297,-0.961538,-1.941748,-1.980198,-1.010101,-2.040816,-1.041667,-2.105263,NaN,NaN
4,Issyk-Kul,NaN,NaN,NaN,-2.173913,0.000000,0.000000,-2.222222,0.000000,-2.272727,0.000000,-2.325581,0.000000,0.000000,-2.380952,0.000000,2.439024,4.761905,2.272727,4.444444,2.127660,4.166667,-6.000000,-6.382979,2.272727,4.444444,2.127660,4.166667,2.000000,3.921569,1.886792,NaN,NaN
5,Jalal-Abad,NaN,NaN,NaN,1.219512,1.204819,2.380952,1.162791,1.149425,1.136364,2.247191,1.098901,1.086957,1.075269,2.127660,1.041667,0.000000,1.030928,0.000000,0.000000,0.000000,1.020408,8.080808,7.476636,-1.739130,-2.654867,-2.727273,-1.869159,-2.857143,-2.941176,-3.030303,NaN,NaN
6,Naryn,NaN,NaN,NaN,0.000000,0.000000,0.000000,-3.703704,0.000000,0.000000,0.000000,-3.846154,0.000000,0.000000,0.000000,-4.000000,4.166667,0.000000,0.000000,0.000000,4.000000,0.000000,3.846154,3.703704,0.000000,3.571429,0.000000,3.448276,0.000000,3.333333,0.000000,NaN,NaN
7,Osh,NaN,NaN,NaN,-2.380952,-1.829268,-1.863354,-2.531646,-1.948052,-2.649007,-2.040816,-2.083333,-2.836879,-2.189781,-2.985075,-2.307692,1.574803,0.775194,1.538462,1.515152,0.746269,1.481481,1.459854,1.438849,2.836879,2.758621,2.684564,2.614379,3.184713,2.469136,2.409639,NaN,NaN
8,Talas,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.761905,4.545455,8.695652,4.000000,3.846154,3.703704,3.571429,3.448276,-3.333333,0.000000,-3.448276,0.000000,-3.571429,0.000000,-3.703704,NaN,NaN


In [42]:
#Drop the NAN values
pop_pct_change.dropna(axis=1, how='any', inplace=True)

# Display the first 10 rows of the percentage change DataFrame
pop_pct_change.round(2)


,Region,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Total,1.19,1.18,1.16,1.15,1.33,1.12,1.29,1.09,1.26,1.96,2.10,2.05,2.01,1.97,1.94,1.90,2.02,1.83
1,Batken,0.00,0.00,0.00,0.00,0.00,2.38,0.00,0.00,0.00,4.65,6.67,-2.08,0.00,-2.13,0.00,0.00,-2.17,0.00
2,Bishkek,5.13,4.88,4.65,-2.22,-1.14,-2.30,-1.18,-1.19,-2.41,-3.70,-3.85,14.67,12.79,11.34,10.19,10.08,8.40,7.75
3,Chuy,0.00,0.00,0.00,3.75,3.61,3.49,2.25,3.30,3.19,4.12,2.97,-0.96,-1.94,-1.98,-1.01,-2.04,-1.04,-2.11
4,Issyk-Kul,0.00,-2.38,0.00,2.44,4.76,2.27,4.44,2.13,4.17,-6.00,-6.38,2.27,4.44,2.13,4.17,2.00,3.92,1.89
5,Jalal-Abad,1.08,2.13,1.04,0.00,1.03,0.00,0.00,0.00,1.02,8.08,7.48,-1.74,-2.65,-2.73,-1.87,-2.86,-2.94,-3.03
6,Naryn,0.00,0.00,-4.00,4.17,0.00,0.00,0.00,4.00,0.00,3.85,3.70,0.00,3.57,0.00,3.45,0.00,3.33,0.00
7,Osh,-2.19,-2.99,-2.31,1.57,0.78,1.54,1.52,0.75,1.48,1.46,1.44,2.84,2.76,2.68,2.61,3.18,2.47,2.41
8,Talas,0.00,0.00,0.00,4.76,4.55,8.70,4.00,3.85,3.70,3.57,3.45,-3.33,0.00,-3.45,0.00,-3.57,0.00,-3.70


In [43]:
# Drop the NAN values
pop.dropna(axis=1, how='any', inplace=True)

# Function to calculate CAGR for each region (row)
def calculate_cagr(row):
    # Skip the first column (Region)
    years = len(row) - 1  # Exclude the Region column from the count
    start_value = row[1]  # Start value (first year data)
    end_value = row[years]  # End value (last year data)

    # If start_value or end_value is 0, return NaN
    if start_value == 0 or end_value == 0:
        return np.nan

    # Calculate CAGR
    try:
        cagr = ((end_value / start_value) ** (1 / years)) - 1
    except ZeroDivisionError:
        return np.nan  # Handle potential divide-by-zero error

    return round(cagr * 100, 2)  # Convert to percentage and round to two decimal places

# Apply the CAGR function to each row of the population DataFrame, excluding the Region column
pop['CAGR'] = pop.iloc[:, 1:].apply(calculate_cagr, axis=1)

# Display the updated DataFrame with the CAGR values
pop.head(10)

<ipython-input-43-7d4b1e2e6ad6>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start_value = row[1]  # Start value (first year data)
<ipython-input-43-7d4b1e2e6ad6>:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_value = row[years]  # End value (last year data)


,Region,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,CAGR
0,Total,5.04,5.10,5.16,5.22,5.28,5.35,5.41,5.48,5.54,5.61,5.72,5.84,5.96,6.08,6.20,6.32,6.44,6.57,6.69,1.52
1,Batken,0.42,0.42,0.42,0.42,0.42,0.42,0.43,0.43,0.43,0.43,0.45,0.48,0.47,0.47,0.46,0.46,0.46,0.45,0.45,0.38
2,Bishkek,0.78,0.82,0.86,0.90,0.88,0.87,0.85,0.84,0.83,0.81,0.78,0.75,0.86,0.97,1.08,1.19,1.31,1.42,1.53,3.53
3,Chuy,0.80,0.80,0.80,0.80,0.83,0.86,0.89,0.91,0.94,0.97,1.01,1.04,1.03,1.01,0.99,0.98,0.96,0.95,0.93,0.84
4,Issyk-Kul,0.42,0.42,0.41,0.41,0.42,0.44,0.45,0.47,0.48,0.50,0.47,0.44,0.45,0.47,0.48,0.50,0.51,0.53,0.54,1.41
5,Jalal-Abad,0.93,0.94,0.96,0.97,0.97,0.98,0.98,0.98,0.98,0.99,1.07,1.15,1.13,1.10,1.07,1.05,1.02,0.99,0.96,0.12
6,Naryn,0.25,0.25,0.25,0.24,0.25,0.25,0.25,0.25,0.26,0.26,0.27,0.28,0.28,0.29,0.29,0.30,0.30,0.31,0.31,1.20
7,Osh,1.37,1.34,1.30,1.27,1.29,1.30,1.32,1.34,1.35,1.37,1.39,1.41,1.45,1.49,1.53,1.57,1.62,1.66,1.70,1.33
8,Talas,0.21,0.21,0.21,0.21,0.22,0.23,0.25,0.26,0.27,0.28,0.29,0.30,0.29,0.29,0.28,0.28,0.27,0.27,0.26,1.19


In [44]:
#need to convert the dataframe csv if I want to download this
pop.to_csv('pop.csv', index=False)

#let's download this to csv for data visualization PS0 or PS1!
files.download('pop.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
#Recap; we see that Bishkek had a CAGR throught he period 2003-2021 of 3.53%. This is more than double the total and any other region.
#that is to say, while the percentages look small, Bishkek has far exceeded the actual percentage growth. the nominal numbers above are in millions.
#So Bishkek doubled in size during this time period. most other places barely saw increases. Talk about everyone being packed into one small space!

In [46]:
#Ok. Now need to convert a raw JSON file with per capita income monthly by region. I don't understand why downloading it led to a weird text file... Chatgpt lead the way!


# let's load the third data set - this was downloaded from Kyrgyzstan bureau of stats
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1kUbVCPCkv5G27fCeEYf_IxOlRqvL6y6i' -O percapita.json
#Read in the raw JSON data directly from file
income = pd.read_json('/content/percapita.json')

# Check the structure of the loaded data to see how it is organized
print(income.head())

# Initialize a list to hold the rows of the DataFrame
rows = []

# Assuming the structure of 'income' is similar to your original JSON structure
# where 'data' is the key containing the region data
for region in income['data']:
    region_name = region['title_en']  # Get the region name
    values = region['values']  # Get the list of values for this region

    # Create a dictionary with region as the first entry and year-value pairs following
    region_data = {'Region': region_name}
    for year_data in values:
        year = year_data['key']
        value = year_data['value']
        region_data[year] = value

    # Append the row to the list
    rows.append(region_data)

# Create a DataFrame from the rows
income_df = pd.DataFrame(rows)
# Reorder the columns so that years are in ascending order (from 2019 to 2023, etc.)
year_columns = sorted([col for col in income_df.columns if isinstance(col, int)])  # Get only the year columns
ordered_columns = ['Region'] + year_columns  # Place 'Region' as the first column, followed by years

# Reorder the DataFrame columns
income_df = income_df[ordered_columns]

# Display the DataFrame
income_df

--2025-01-27 23:52:02--  https://drive.google.com/uc?export=download&id=1kUbVCPCkv5G27fCeEYf_IxOlRqvL6y6i
Resolving drive.google.com (drive.google.com)... 173.194.194.113, 173.194.194.138, 173.194.194.100, ...
Connecting to drive.google.com (drive.google.com)|173.194.194.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1kUbVCPCkv5G27fCeEYf_IxOlRqvL6y6i&export=download [following]
--2025-01-27 23:52:02--  https://drive.usercontent.google.com/download?id=1kUbVCPCkv5G27fCeEYf_IxOlRqvL6y6i&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.214.132, 2607:f8b0:4001:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.214.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3378 (3.3K) [application/octet-stream]
Saving to: ‘percapita.json’

percapita.json      100%[===================>]   3.30K  --.-K

,Region,2019,2020,2021,2022,2023
0,Kyrgyz Republic,5684.7,5625.4,6647.8,7948.4,9716.0
1,Batken oblast,5346.5,5386.9,6205.4,7531.8,9877.4
2,Jalal-Abat oblast,5293.2,5415.0,6672.2,8253.9,10327.6
3,Yssyk-Kul oblast,5719.5,6433.8,7173.2,9114.5,10105.3
4,Naryn oblast,4663.7,4588.5,5808.8,7096.2,8290.5
5,Osh oblast (until 2013 y. including Osh city),5914.4,5700.5,6922.4,7828.4,9512.8
6,Talas oblast,4834.5,5262.9,5732.5,6986.7,8406.2
7,Chui oblast,6116.3,5774.7,6755.1,7975.8,9298.0
8,Bishkek city,6340.8,6012.9,6946.2,8484.1,10767.1
9,Osh city,4892.3,4770.0,5523.1,5919.1,7581.1


In [47]:
# Define the exchange rate
exchange_rate = 87.45

# Convert the whole DataFrame from KGS to USD
income_df_usd = income_df.copy()

# Apply the conversion for each year (for all regions)
for year in income_df.columns[1:]:  # Skip the 'Region' column
    income_df_usd[year] = income_df[year] / exchange_rate

# Display the converted DataFrame
income_df_usd.round(2).head(10)

,Region,2019,2020,2021,2022,2023
0,Kyrgyz Republic,65.01,64.33,76.02,90.89,111.10
1,Batken oblast,61.14,61.60,70.96,86.13,112.95
2,Jalal-Abat oblast,60.53,61.92,76.30,94.38,118.10
3,Yssyk-Kul oblast,65.40,73.57,82.03,104.23,115.56
4,Naryn oblast,53.33,52.47,66.42,81.15,94.80
5,Osh oblast (until 2013 y. including Osh city),67.63,65.19,79.16,89.52,108.78
6,Talas oblast,55.28,60.18,65.55,79.89,96.13
7,Chui oblast,69.94,66.03,77.25,91.20,106.32
8,Bishkek city,72.51,68.76,79.43,97.02,123.12
9,Osh city,55.94,54.55,63.16,67.69,86.69


In [48]:
#Code to calculate statistics for one year (2023) - monthly salary in USD per capita
# create "selection" for 2023
year_2023 = income_df_usd[2023]

# Calculate statistics for the selected year, defined as below
statistics_2023 = {
    'mean': year_2023.mean(),
    'median': year_2023.median(),
    'min': year_2023.min(),
    'max': year_2023.max()
}

statistics_2023
#yikes! per capita income is super low, ranging from 86 to 123 bucks.
#interesting to note that from a quick  search, it looks like the average wage is closer to double this
#Either this means that 1) ministry data is poor 2) there's a large non working part of the population between youth and adults
#(40% of nation under <18)that are captured in this metric, with about 10% above 60. this could be a good thing to dig further into!


{'mean': 107.35506003430532,
 'median': 109.94168096054888,
 'min': 86.6906803887936,
 'max': 123.12292738707833}

In [49]:
from google.colab import files
#convert dataframe to csv
income_df_usd.to_csv('income_usd.csv', index=False)

# Download the file, so I can use this for visual!
files.download('income_usd.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>